# Model Training

In [1]:
# Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Dataset
total_data = pd.read_csv('../data/interim/clean_match_data.csv')
total_data['members_team_1'] = total_data['members_team_1'].apply(eval)
total_data['members_team_2'] = total_data['members_team_2'].apply(eval)
total_data.head()

,members_team_1,members_team_2,map,avg_adr_team_1,sum_assists_team_1,sum_deaths_team_1,sum_fkdiffs_team_1,sum_hs_team_1,mean_kdratio_team_1,sum_kills_team_1,avg_adr_team_2,sum_assists_team_2,sum_deaths_team_2,sum_fkdiffs_team_2,sum_hs_team_2,mean_kdratio_team_2,sum_kills_team_2,winning_team
0,"(168, 5737, 2492, 10814, 11247)","(5698, 10563, 2532, 7382, 5736)",Train,58.720000,4.0,89.0,-6.0,24.0,0.47000,48.0,92.000000,15.0,48.0,6.0,36.0,0.820000,89.0,2
1,"(483, 484, 2757, 3347, 7594)","(2469, 7398, 7592, 4954, 429)",Dust2,72.598373,21.0,54.0,12.0,32.0,0.69258,89.0,75.954746,15.0,90.0,-12.0,25.0,0.709509,54.0,1
2,"(10565, 11302, 10795, 10797, 10798)","(5698, 10563, 2532, 7382, 5736)",Cache,62.880000,11.0,96.0,-5.0,28.0,0.53600,66.0,81.040000,17.0,66.0,5.0,33.0,0.792000,96.0,2
3,"(12272, 10897, 12102, 8493, 11230)","(9571, 10372, 8708, 9705, 9069)",Mirage,69.660000,13.0,102.0,3.0,45.0,0.63740,86.0,79.900000,26.0,86.0,-3.0,51.0,0.689200,102.0,2
4,"(483, 484, 2757, 3347, 7594)","(338, 7796, 472, 1866, 7403)",Train,87.800000,19.0,65.0,3.0,39.0,0.81760,96.0,66.640000,18.0,96.0,-3.0,31.0,0.582600,65.0,1


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Concatenate, Embedding, Flatten, concatenate

X = total_data[[
    'members_team_1',
    'members_team_2',
    'map',  
    'avg_adr_team_1',
    'sum_assists_team_1', 
    'sum_deaths_team_1', 
    'sum_fkdiffs_team_1',
    'sum_hs_team_1', 
    'mean_kdratio_team_1', 
    'sum_kills_team_1',     
    'avg_adr_team_2', 
    'sum_assists_team_2',
    'sum_deaths_team_2', 
    'sum_fkdiffs_team_2', 
    'sum_hs_team_2',
    'mean_kdratio_team_2', 
    'sum_kills_team_2'
]]

y = total_data['winning_team']

numerical_features = [
    'avg_adr_team_1',
    'sum_assists_team_1', 
    'sum_deaths_team_1', 
    'sum_fkdiffs_team_1',
    'sum_hs_team_1', 
    'mean_kdratio_team_1', 
    'sum_kills_team_1',     
    'avg_adr_team_2', 
    'sum_assists_team_2',
    'sum_deaths_team_2', 
    'sum_fkdiffs_team_2', 
    'sum_hs_team_2',
    'mean_kdratio_team_2', 
    'sum_kills_team_2']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69, stratify=y)





2024-02-01 01:08:40.410684: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-01 01:08:40.449989: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 01:08:40.450020: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 01:08:40.451437: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 01:08:40.457336: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-01 01:08:40.457888: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:

X_train_frozenset_1 = X_train[['members_team_1']]
X_test_frozenset_1 = X_test[['members_team_1']]

X_train_frozenset_2 = X_train[['members_team_2']]
X_test_frozenset_2 = X_test[['members_team_2']]

# Standardize numerical features
scaler = StandardScaler()
X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test[numerical_features] = scaler.transform(X_test[numerical_features])

# One-hot encode categorical features
encoder_categorical = OneHotEncoder(sparse=False, drop='first')
X_train['map'] = encoder_categorical.fit_transform(X_train[['map']])
X_test['map'] = encoder_categorical.transform(X_test[['map']])

# Determine the number of categories dynamically
num_categories = len(total_data['members_team_1'].unique()) + len(total_data['members_team_2'].unique())

# Use a hash function to map frozensets to integers
hash_function = lambda x: hash(frozenset(x))
X_train['members_team_1'] = X_train_frozenset_1['members_team_1'].apply(hash_function)
X_test['members_team_1'] = X_test_frozenset_1['members_team_1'].apply(hash_function)

X_train['members_team_2'] = X_train_frozenset_2['members_team_2'].apply(hash_function)
X_test['members_team_2'] = X_test_frozenset_2['members_team_2'].apply(hash_function)




/home/alamo/.pyenv/versions/3.11.5/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [4]:
# model = Sequential()
embedding_dim = 32

# # Add an Embedding layer as the input layer
# model.add(Embedding(input_dim=num_categories, output_dim=embedding_dim, input_length=2))

# # Add other layers to your model (e.g., Flatten, Dense)
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [5]:
# Input layers for each branch
input1 = Input(shape=(2,))
input2 = Input(shape=(len(X_train.columns)-2,))

embedding_layer1 = Embedding(input_dim=num_categories, output_dim=embedding_dim)(input1)
flatten1 = Flatten()(embedding_layer1)

# Dense layer for the second input
dense_layer2 = Dense(32, activation='relu')(input2)
flatten2 = Flatten()(dense_layer2)

# Concatenate the flattened layers
concatenated = concatenate([flatten1, flatten2])

# Additional layers for further processing
dense1 = Dense(128, activation='relu')(concatenated)
output_layer = Dense(1, activation='sigmoid')(dense1)

# Create the model
model = Model(inputs=[input1, input2], outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 2)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 15)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 2, 32)                837696    ['input_1[0][0]']             
                                                                                                  
 dense (Dense)               (None, 32)                   512       ['input_2[0][0]']             
                                                                                              

2024-02-01 01:08:45.489976: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-01 01:08:45.490440: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [107]:
# # Embedding layer for frozenset feature
# embedding_dim = 8  # Adjust this based on your problem
# main_input = Input(shape=(2,), name='main_input')
# embedding_layer = Embedding(input_dim=num_categories, output_dim=embedding_dim, input_length=1)(main_input)
# flattened_embedding = Flatten()(embedding_layer)

# # Prepare model
# other_input = Input(shape=(len(X_train.columns)-2,), name='other_input')

# x = Dense(64, activation='relu')(flattened_embedding)
# x = Dense(32, activation='relu')(x)

# merged = Concatenate()([x, other_input])

# output_layer = Dense(1, activation='linear', name='output')(merged)

# # Create the model
# model = Model(inputs=[main_input, other_input], outputs=output_layer)

# # Compile the model
# model.compile(optimizer='adam', loss='mean_squared_error')


In [7]:
X_train

,members_team_1,members_team_2,map,avg_adr_team_1,sum_assists_team_1,sum_deaths_team_1,sum_fkdiffs_team_1,sum_hs_team_1,mean_kdratio_team_1,sum_kills_team_1,avg_adr_team_2,sum_assists_team_2,sum_deaths_team_2,sum_fkdiffs_team_2,sum_hs_team_2,mean_kdratio_team_2,sum_kills_team_2
18530,4580754493439845793,4897923747383815096,0.0,-0.186473,-0.637478,-0.588400,0.738440,1.137472,-0.069007,0.628200,0.073382,0.011769,0.616711,-0.731075,-0.989075,0.174407,-0.624171
13838,637201321473331546,2389634046530821950,0.0,0.177751,-0.091778,0.016925,-0.088203,-0.843119,-0.318415,0.486066,-0.424023,-0.904181,0.474699,0.095568,0.043029,-0.890000,-0.017650
83278,4335187388067358377,7818529998147922221,0.0,0.911294,1.681748,-0.026312,-1.080174,0.194333,0.660283,0.628200,0.068728,-0.380781,0.616711,1.087540,-0.426109,0.157818,-0.060973
38666,6755149991715981064,-7179575996970549779,0.0,0.456673,0.999623,1.962614,-0.088203,0.194333,0.410726,2.049540,0.757709,1.058568,2.036837,0.095568,1.356615,0.329660,1.975203
55555,7614586807637577173,4228947456739426404,0.0,-0.285207,0.863198,1.443764,1.399754,1.609041,-0.011067,1.907406,0.097995,1.712818,1.894825,-1.392389,0.230684,0.255349,1.455328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76180,5402653557813088469,-4531372644324569646,0.0,-1.262981,-0.773903,0.406063,-0.584188,-0.088608,-0.719877,-0.414116,-0.205992,0.011769,-0.424714,0.591554,0.230684,0.195540,0.415579
54958,5740322664991383492,-8717073363692781851,0.0,-1.186113,-1.319603,0.233113,-1.080174,0.194333,-0.630904,-0.508872,-0.201631,-0.119081,-0.519390,1.087540,-0.707592,0.830517,0.242287
15146,-8747395246827170069,-6711096841013417563,1.0,2.343240,-1.728878,-2.317901,1.399754,-0.654491,1.845137,-0.366738,-2.351424,-1.689280,-0.330039,-1.392389,-2.396489,-1.998591,-2.313764
35381,-3273508006688049986,-5395851614749566366,0.0,-2.163438,-2.138153,0.103401,-0.749517,-2.352141,-3.154687,-2.261858,2.015752,-0.249931,-2.270878,0.756883,-0.238454,1.624763,0.112318


In [18]:


# Train the model
model.fit([X_test[['members_team_1','members_team_2']], pd.concat([X_test['map'], X_test[numerical_features]], axis=1)], y_train, epochs=50, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
loss = model.evaluate(X_test[['members_team_1']], y_test)
print(f'Mean Squared Error on Test Set: {loss}')

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node model/embedding/embedding_lookup defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/alamo/.pyenv/versions/3.11.5/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/alamo/.pyenv/versions/3.11.5/lib/python3.11/site-packages/traitlets/config/application.py", line 1046, in launch_instance

  File "/home/alamo/.pyenv/versions/3.11.5/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/home/alamo/.pyenv/versions/3.11.5/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/home/alamo/.pyenv/versions/3.11.5/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/home/alamo/.pyenv/versions/3.11.5/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/home/alamo/.pyenv/versions/3.11.5/lib/pyt